# Planet API Python Client


This tutorial is an introduction to [Planet](https://www.planet.com)'s Data API using the official [Python client](https://github.com/planetlabs/planet-client-python), the `planet` module.

## Requirements

This tutorial assumes familiarity with the [Python](https://python.org) programming language throughout. Python modules used in this tutorial are:
* [IPython](https://ipython.org/) and [Jupyter](https://jupyter.org/)
* [planet](https://github.com/planetlabs/planet-client-python)
* [geojsonio](https://pypi.python.org/pypi/geojsonio)

You should also have an account on the Planet Platform and retrieve your API key from your [account page](https://www.planet.com/account/).

## Useful links 
* [Documentation](https://planetlabs.github.io/planet-client-python/index.html)
* [Planet Data API reference](https://www.planet.com/docs/reference/data-api/)

This tutorial will cover the basic operations possible with the Python client, particularly those that interact with the Data API.

The basic workflow for interacting with the Data API is:
1. search item types based on filters
1. activate assets
1. download assets

## Set up

In order to interact with the Planet API using the client, we need to import the necessary packages & define helper functions.

In [37]:
from datetime import datetime
from planet import Auth
from planet import Session, DataClient, OrdersClient
import json
import requests
import time
import os

# We will also create a small helper function to print out JSON with proper indentation.
def p(data):
    print(json.dumps(data, indent=2))

We next need to create a `client` object registered with our API key. The API key will be automatically read from the `PL_API_KEY` environment variable if it exists. If not, you can provide it below. 

In [54]:
# if your Planet API Key is not set as an environment variable, you can paste it below
if os.environ.get('PL_API_KEY', ''):
    API_KEY = os.environ.get('PL_API_KEY', '')
else:
    API_KEY = 'PASTE_YOUR_API_KEY_HERE'

client = Auth.from_key(API_KEY)

# Setup the session
session = requests.Session()

# Authenticate
session.auth = (API_KEY, "")

Let's also read in a GeoJSON geometry into a variable so we can use it during testing.

In [7]:
with open("data/san-francisco.json") as f:
    geom = json.loads(f.read())

## Searching

We can search for items that are interesting by using the `quick_search` member function. Searches, however, always require a proper request that includes a filter that selects the specific items to return as seach results.

### Filters

The possible filters include `and_filter`, `date_filter`, `range_filter` and so on, mirroring the options supported by the Planet API.


In [9]:
# Define the filters we'll use to find our data

item_types = ["REOrthoTile", "PSOrthoTile"]

geom_filter = {
   "type":"GeometryFilter",
   "field_name":"geometry",
   "config":geom
}

cloud_cover_filter = {
"type":"RangeFilter",
"field_name":"cloud_cover",
"config":{
  "lt":0.1 }
}

date_range_filter = {
"type":"DateRangeFilter",
"field_name":"acquired",
"config":{
  "gt":"2017-01-01T00:00:00Z"}
}

combined_filter = {
"type":"AndFilter",
"config":[
    geom_filter,
    date_range_filter,
    cloud_cover_filter]
}

In [10]:
p(combined_filter)

{
  "type": "AndFilter",
  "config": [
    {
      "type": "GeometryFilter",
      "field_name": "geometry",
      "config": {
        "type": "Polygon",
        "coordinates": [
          [
            [
              -122.47455596923828,
              37.810326435534755
            ],
            [
              -122.49172210693358,
              37.795406713958236
            ],
            [
              -122.52056121826172,
              37.784282779035216
            ],
            [
              -122.51953124999999,
              37.6971326434885
            ],
            [
              -122.38941192626953,
              37.69441603823106
            ],
            [
              -122.38872528076173,
              37.705010235842614
            ],
            [
              -122.36228942871092,
              37.70935613533687
            ],
            [
              -122.34992980957031,
              37.727280276860036
            ],
            [
              -122.3777

Now let's build the request:

In [11]:
async with Session() as sess:
    cl = DataClient(sess)
    request = await cl.create_search(name='planet_client_demo',search_filter=combined_filter, item_types=item_types)

In [12]:
p(request)

{
  "__daily_email_enabled": false,
  "_links": {
    "_self": "https://api.planet.com/data/v1/searches/c1a895f21a5846c583d2387c66d8fa1b",
    "results": "https://api.planet.com/data/v1/searches/c1a895f21a5846c583d2387c66d8fa1b/results"
  },
  "created": "2022-08-23T19:54:39.584254Z",
  "filter": {
    "config": [
      {
        "config": {
          "coordinates": [
            [
              [
                -122.47455596923828,
                37.810326435534755
              ],
              [
                -122.49172210693358,
                37.795406713958236
              ],
              [
                -122.52056121826172,
                37.784282779035216
              ],
              [
                -122.51953124999999,
                37.6971326434885
              ],
              [
                -122.38941192626953,
                37.69441603823106
              ],
              [
                -122.38872528076173,
                37.705010235842614
     

In [46]:
# Search the Data API
async with Session() as sess:
    cl = DataClient(sess)
    items = await cl.run_search(search_id=request['id'])
    item_list = [i async for i in items]

Now, we can iterate through our search results.

In [47]:
for item in item_list:
    print(item['id'], item['properties']['item_type'])

5874254_1056516_2022-08-22_247d PSOrthoTile
5874254_1056416_2022-08-22_247d PSOrthoTile
5874249_1056517_2022-08-22_24a5 PSOrthoTile
5874249_1056516_2022-08-22_24a5 PSOrthoTile
5874249_1056417_2022-08-22_24a5 PSOrthoTile
5874249_1056416_2022-08-22_24a5 PSOrthoTile
5872070_1056517_2022-08-21_241d PSOrthoTile
5872070_1056417_2022-08-21_241d PSOrthoTile
5869395_1056516_2022-08-20_24a4 PSOrthoTile
5869395_1056517_2022-08-20_24a4 PSOrthoTile
5869395_1056417_2022-08-20_24a4 PSOrthoTile
5866875_1056517_2022-08-19_2427 PSOrthoTile
5866875_1056417_2022-08-19_2427 PSOrthoTile
5864323_1056517_2022-08-18_2480 PSOrthoTile
5864323_1056417_2022-08-18_2480 PSOrthoTile
5862695_1056517_2022-08-17_2490 PSOrthoTile
5860470_1056517_2022-08-16_248e PSOrthoTile
5860470_1056516_2022-08-16_248e PSOrthoTile
5860470_1056417_2022-08-16_248e PSOrthoTile
5858188_1056517_2022-08-15_247e PSOrthoTile
5858188_1056516_2022-08-15_247e PSOrthoTile
5858188_1056417_2022-08-15_247e PSOrthoTile
5858188_1056416_2022-08-15_247e 

If the number of items requested is more than 250, the client will automatically fetch more pages of results in order to get the exact number requested.

In [48]:
with open('output/results.json','w') as f:
    jsonStr = json.dumps(item_list)
    f.write(jsonStr)
    f.close()

This GeoJSON file can be opened and viewed in any compatible application.

## Assets and downloads

After a search returns results, the Python client can be used to check for assets and initiate downloads.

The list of assets for an item that a user has access to can be retrieved with `permissions`

In [49]:
print(item['id'])

5801853_1056416_2022-07-21_2486


In [50]:
item['_permissions']

['assets.analytic:download',
 'assets.analytic_5b:download',
 'assets.analytic_5b_xml:download',
 'assets.analytic_dn:download',
 'assets.analytic_dn_xml:download',
 'assets.analytic_sr:download',
 'assets.analytic_xml:download',
 'assets.udm:download',
 'assets.udm2:download',
 'assets.visual:download',
 'assets.visual_xml:download']

In [55]:
# Get the assets link for the item
assets_url = item["_links"]["assets"]

# Print the assets link
print(assets_url)

https://api.planet.com/data/v1/item-types/PSOrthoTile/items/5801853_1056416_2022-07-21_2486/assets/


In [56]:
# Send a GET request to the assets url for the item (Get the list of available assets for the item)
res = session.get(assets_url)

# Assign a variable to the response
assets = res.json()

In [58]:
# Assign a variable to the visual asset from the item's assets
analytic_asset = assets["analytic"]

# Print the visual asset data
p(analytic_asset)

{
  "_links": {
    "_self": "https://api.planet.com/data/v1/assets/eyJpIjogIjU4MDE4NTNfMTA1NjQxNl8yMDIyLTA3LTIxXzI0ODYiLCAiYyI6ICJQU09ydGhvVGlsZSIsICJ0IjogImFuYWx5dGljIiwgImN0IjogIml0ZW0tdHlwZSJ9",
    "activate": "https://api.planet.com/data/v1/assets/eyJpIjogIjU4MDE4NTNfMTA1NjQxNl8yMDIyLTA3LTIxXzI0ODYiLCAiYyI6ICJQU09ydGhvVGlsZSIsICJ0IjogImFuYWx5dGljIiwgImN0IjogIml0ZW0tdHlwZSJ9/activate",
    "type": "https://api.planet.com/data/v1/asset-types/analytic"
  },
  "_permissions": [
    "download"
  ],
  "md5_digest": null,
  "status": "inactive",
  "type": "analytic"
}


In [60]:
# Setup the activation url for a particular asset (in this case the basic_analytic_4b asset)
activation_url = analytic_asset["_links"]["activate"]

# Send a request to the activation url to activate the item
res = session.get(activation_url)

# Print the response from the activation request
p(res.status_code)

202


A response of 202 means that the request has been accepted and the activation will begin shortly. A 204 code indicates that the asset is already active and no further action is needed. A 401 code means the user does not have permissions to download this file.

Below, we are polling the API until the item is done activation. This may take awhile.

In [63]:
asset_activated = False

while asset_activated == False:
    # Send a request to the item's assets url
    res = session.get(assets_url)

    # Assign a variable to the item's assets url response
    assets = res.json()

    # Assign a variable to the basic_analytic_4b asset from the response
    visual = assets["analytic"]

    asset_status = analytic_asset["status"]
    print(asset_status)
    
    # If asset is already active, we are done
    if asset_status == 'active':
        asset_activated = True
        print("Asset is active and ready to download")

# Print the ps3b_analytic asset data    
p(analytic_asset)

inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
i

inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
i

inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
i

inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
i

inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
inactive
i

KeyboardInterrupt: 

In [20]:
callback = api.write_to_file(directory='output/')
body = client.download(assets['analytic_xml'], callback=callback)
print(body)

## Saved Searches

The Python API client can also help in managing saved searches on the Planet Platform.

In [22]:
searches = client.get_searches()

View your saved searches:

In [36]:
for search in searches.items_iter(100):
    print(search['id'], search['name'])

a7a648fca59245bca42d2466b289f34f San Francisco
6950887354d84e23889c3e308355fdcf Vancouver Island
fe44f67590ae44d6a4e5e8030de85f90 Vancouver Island


Filter saved searches by location and by band, etc:

In [24]:
item_types = ["PSScene3Band"]
san_francisco_filter = filters.geom_filter(geom)
req = filters.build_search_request(san_francisco_filter, item_types, name="San Francisco")

In [25]:
p(req)

{
  "item_types": [
    "PSScene3Band"
  ],
  "filter": {
    "field_name": "geometry",
    "type": "GeometryFilter",
    "config": {
      "type": "Polygon",
      "coordinates": [
        [
          [
            -122.47455596923828,
            37.810326435534755
          ],
          [
            -122.49172210693358,
            37.795406713958236
          ],
          [
            -122.52056121826172,
            37.784282779035216
          ],
          [
            -122.51953124999999,
            37.6971326434885
          ],
          [
            -122.38941192626953,
            37.69441603823106
          ],
          [
            -122.38872528076173,
            37.705010235842614
          ],
          [
            -122.36228942871092,
            37.70935613533687
          ],
          [
            -122.34992980957031,
            37.727280276860036
          ],
          [
            -122.37773895263672,
            37.76230130281876
          ],
          [


In [26]:
res = client.create_search(req)

In [27]:
search = res.get()
print(search["id"], search["name"])

ae24a8d7251244b1ac283bc5b9398b78 San Francisco


In [28]:
res = client.saved_search(search["id"])

Examine view angles for every item in the search results:

In [29]:
for item in res.items_iter(20):
    print(item["id"], item["properties"]["view_angle"])

20211018_182142_64_2231 5
20211018_182140_43_2231 5
20211017_180741_66_2463 5
20211017_180739_36_2463 5
20211017_183250_103b 2
20211017_183249_103b 1.9
20211015_185028_06_2416 3
20211015_185025_58_2416 3
20211015_180740_54_2206 5.1
20211015_180738_33_2206 5.1
20211015_183336_1005 1
20211015_183334_1005 1
20211015_183335_1005 1
20211015_180900_61_2432 5.1
20211014_180933_17_2223 5.1
20211014_180930_96_2223 5.1
20211014_180928_75_2223 5
20211013_180324_31_2449 3
20211013_180322_00_2449 3
20211013_180319_70_2449 3


# Statistics

The Python API client can also help report statistical summaries of the amount of data in the Planet API.

In [30]:
item_types = ["PSScene3Band"]
san_francisco_filter = filters.geom_filter(geom)
req = filters.build_search_request(san_francisco_filter, item_types, interval="year")

In [31]:
stats = client.stats(req).get()

In [32]:
p(stats)

{
  "buckets": [
    {
      "count": 3,
      "start_time": "2014-01-01T00:00:00.000000Z"
    },
    {
      "count": 60,
      "start_time": "2015-01-01T00:00:00.000000Z"
    },
    {
      "count": 200,
      "start_time": "2016-01-01T00:00:00.000000Z"
    },
    {
      "count": 1072,
      "start_time": "2017-01-01T00:00:00.000000Z"
    },
    {
      "count": 1637,
      "start_time": "2018-01-01T00:00:00.000000Z"
    },
    {
      "count": 1452,
      "start_time": "2019-01-01T00:00:00.000000Z"
    },
    {
      "count": 1434,
      "start_time": "2020-01-01T00:00:00.000000Z"
    },
    {
      "count": 1373,
      "start_time": "2021-01-01T00:00:00.000000Z"
    }
  ],
  "interval": "year",
  "utc_offset": "+0h"
}


In [33]:
assets = client.get_assets(item)

In [34]:
assets.last_modified()

In [35]:
assets.get()

{'analytic': {'_links': {'_self': 'https://api.planet.com/data/v1/assets/eyJpIjogIjIwMjExMDEzXzE4MDMxOV83MF8yNDQ5IiwgImMiOiAiUFNTY2VuZTNCYW5kIiwgInQiOiAiYW5hbHl0aWMiLCAiY3QiOiAiaXRlbS10eXBlIn0',
   'activate': 'https://api.planet.com/data/v1/assets/eyJpIjogIjIwMjExMDEzXzE4MDMxOV83MF8yNDQ5IiwgImMiOiAiUFNTY2VuZTNCYW5kIiwgInQiOiAiYW5hbHl0aWMiLCAiY3QiOiAiaXRlbS10eXBlIn0/activate',
   'type': 'https://api.planet.com/data/v1/asset-types/analytic'},
  '_permissions': ['download'],
  'md5_digest': None,
  'status': 'inactive',
  'type': 'analytic'},
 'analytic_dn': {'_links': {'_self': 'https://api.planet.com/data/v1/assets/eyJpIjogIjIwMjExMDEzXzE4MDMxOV83MF8yNDQ5IiwgImMiOiAiUFNTY2VuZTNCYW5kIiwgInQiOiAiYW5hbHl0aWNfZG4iLCAiY3QiOiAiaXRlbS10eXBlIn0',
   'activate': 'https://api.planet.com/data/v1/assets/eyJpIjogIjIwMjExMDEzXzE4MDMxOV83MF8yNDQ5IiwgImMiOiAiUFNTY2VuZTNCYW5kIiwgInQiOiAiYW5hbHl0aWNfZG4iLCAiY3QiOiAiaXRlbS10eXBlIn0/activate',
   'type': 'https://api.planet.com/data/v1/asset-types/analyt